In [1]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('/Users/gianmarcoferrara/ComplexNetworksProject/temporal_series/temporal_series_1.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Appadjust max_corr based on tau direction
    if best_tau >= 0:
        return max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "2_1_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

#filename_sub_2 = "2_3_sub_mat_2.csv"
#compute_submatrix(data, group_2, filename_sub_2, max_tau)

#filename_sub_3 = "2_3_sub_mat_3.csv"
#compute_submatrix(data, group_3, filename_sub_3, max_tau)

#filename_cross_1 = "2_3_cross_matrix_1_2.csv"
#compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

#filename_cross_2 = "2_3_cross_matrix_1_3.csv"
#compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

#filename_cross_3 = "2_3_cross_matrix_2_3.csv"
#compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 2_1_sub_mat_1.csv has been saved.


,1001.0,1003.0,1005.0,1007.0,1009.0,1011.0,1013.0,1015.0,1017.0,1019.0,...,21063.0,21065.0,21067.0,21069.0,21071.0,21073.0,21075.0,21077.0,21079.0,21081.0
1001.0,1.0,0.529642,0.534112,0,0.627929,0.369869,0,0,0.258392,0.535817,...,0.328997,0,0.531967,0,0.276971,0,0.457578,0.246939,0.457867,0.145796
1003.0,0.529642,1.0,0.503745,0,0.798915,0,0,0.835932,0,0,...,0.3136,0.222267,0.587643,0,0.374177,0,0,0,0,0.173159
1005.0,0.534112,0.503745,1.0,0.565358,0.599503,0.382276,0,0.521075,0.176476,0.533845,...,0,0.338253,0.553554,0.327193,0.278934,0,0.478909,0,0,0
1007.0,0,0,0.565358,1.0,0.59114,0,0,0.555181,0,0.55514,...,0.351542,0,0.494281,0.58044,0.343274,0,0.384711,0,0.468538,0.196867
1009.0,0.627929,0.798915,0.599503,0.59114,1.0,0,0,0,0,0.571054,...,0,0,0,0,0.36576,0,0,0,0,0.234229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21073.0,0,0,0,0,0,0.455812,0.266825,0,0,0,...,0,0,0,0,0,1.0,0,0,0.432696,0
21075.0,0.457578,0,0.478909,0.384711,0,0.4186,0.331682,0,0,0,...,0,0,0,0.452233,0,0,1.0,0,0.469301,0
21077.0,0.246939,0,0,0,0,0.54452,0.393887,0,0,0,...,0,0,0.326261,0,0.209495,0,0,1.0,0.334856,0
21079.0,0.457867,0,0,0.468538,0,0.262719,0.143241,0,0.267511,0,...,0,0,0.448851,0,0,0.432696,0.469301,0.334856,1.0,0.188356
